In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import trange
import random
import math
from scipy import interp
import statistics 
import os
from tqdm import tqdm

from tcrmodels.ergo2.model import ERGO2
from tcrmodels.nettcr2.model import NetTCR2
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, auc

from matplotlib import collections
from matplotlib import colors
from numpy.random import normal

Using TensorFlow backend.


In [2]:
metrics = [
    'AUROC',
    'Accuracy',
    'Recall',
    'Precision',
    'F1 score',
    'AUPR'
]

def pr_auc(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall, precision)
    return pr_auc

def get_scores(y_true, y_prob, y_pred):
    """
    Compute a df with all classification metrics and respective scores.
    """
    
    scores = [
        roc_auc_score(y_true, y_prob),
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        f1_score(y_true, y_pred),
        pr_auc(y_true, y_prob)
    ]
    
    df = pd.DataFrame(data={'score': scores, 'metrics': metrics})
    return df

In [3]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)

In [4]:
login = os.getlogin( )
DATA_BASE = f"/home/{login}/Git/tc-hard/tc-hard-data/"
RESULTS_BASE = f"/home/{login}/Git/tc-hard/notebooks/notebooks.classification/results/"

In [5]:
import pickle as p

with open(DATA_BASE + "ergo2-paper/mcpas_train_samples.pickle", "rb") as f:
    mcpas_train = pd.DataFrame(p.load(f))

    
with open(DATA_BASE + "ergo2-paper/mcpas_test_samples.pickle", "rb") as f:
    mcpas_test = pd.DataFrame(p.load(f))

mcpas = pd.concat([mcpas_train, mcpas_test]).reset_index(drop=True)

mcpas = mcpas[["tcrb", "peptide", "sign"]].drop_duplicates(subset=["tcrb", "peptide"], keep=False).dropna().reset_index(drop=True)
print('pos:', len(mcpas[mcpas['sign']==1]))
print('neg:', len(mcpas[mcpas['sign']==0]))

pos: 8612
neg: 47630


In [6]:
import pickle as p

with open(DATA_BASE + "ergo2-paper/vdjdb_train_samples.pickle", "rb") as f:
    vdjdb_train = pd.DataFrame(p.load(f))

    
with open(DATA_BASE + "ergo2-paper/vdjdb_test_samples.pickle", "rb") as f:
    vdjdb_test = pd.DataFrame(p.load(f))

vdjdb = pd.concat([vdjdb_train, vdjdb_test]).reset_index(drop=True)

vdjdb = vdjdb[["tcrb", "peptide", "sign"]].drop_duplicates(subset=["tcrb", "peptide"], keep=False).dropna().reset_index(drop=True)
print('pos:', len(vdjdb[vdjdb['sign']==1]))
print('neg:', len(vdjdb[vdjdb['sign']==0]))

pos: 17534
neg: 106093


# NetTCR-2.0

In [13]:
results_nettcr2 = []

df_train = vdjdb.copy()
df_test = mcpas.copy()

max_pep_len = max(df_train["peptide"].str.len().max(), df_test["peptide"].str.len().max())
max_cdr3b_len = max(df_train["tcrb"].str.len().max(), df_test["tcrb"].str.len().max())


model = NetTCR2(
    architecture="b", 
    single_chain_column='tcrb',
    peptide_column='peptide',
    label_column='sign',
    max_pep_len=max_pep_len, 
    max_cdr3_len=max_cdr3b_len
)
model.train(df_train, epochs=1000);

prediction_df = model.test(df_test)

scores_df = get_scores(
    y_true=prediction_df['sign'].to_numpy(), 
    y_prob=prediction_df['prediction'].to_numpy(),
    y_pred=prediction_df['prediction'].to_numpy().round(),
)
results_nettcr2.append(scores_df)
df_test['prediction'] = prediction_df['prediction']
df_test.to_csv(RESULTS_BASE+f"nettcr2.vdjdb2mcpas.csv", index=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1000
123627/123627 [==============================] - 6s 49us/step - loss: 0.4027
Epoch 2/1000
123627/123627 [==============================] - 5s 44us/step - loss: 0.3972
Epoch 3/1000
123627/123627 [==============================] - 6s 45us/step - loss: 0.3954
Epoch 4/1000
123627/123627 [==============================] - 5s 43us/step - loss: 0.3934
Epoch 5/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.3901
Epoch 6/1000
123627/123627 [==============================] - 5s 42us/step - loss: 0.3865
Epoch 7/1000
123627/123627 [==============================] - 5s 42us/step - loss: 0.3812
Epoch 8/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.3746
Epoch 9/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.3701
Epoch 10/1000


123627/123627 [==============================] - 5s 40us/step - loss: 0.2848
Epoch 170/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2847
Epoch 171/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2845
Epoch 172/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2840
Epoch 173/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2841
Epoch 174/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2838
Epoch 175/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2838
Epoch 176/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2835
Epoch 177/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2836
Epoch 178/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2829
Epoch 179/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2828
Epo

123627/123627 [==============================] - 5s 40us/step - loss: 0.2619
Epoch 348/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2611
Epoch 349/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2611
Epoch 350/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2611
Epoch 351/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2611
Epoch 352/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2606
Epoch 353/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2609
Epoch 354/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2606
Epoch 355/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2606
Epoch 356/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2605
Epoch 357/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2609
Epo

123627/123627 [==============================] - 5s 42us/step - loss: 0.2490
Epoch 526/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2494
Epoch 527/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2491
Epoch 528/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2488
Epoch 529/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2490
Epoch 530/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2490
Epoch 531/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2490
Epoch 532/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2488
Epoch 533/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2485
Epoch 534/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2483
Epoch 535/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2485
Epo

123627/123627 [==============================] - 5s 40us/step - loss: 0.2414
Epoch 704/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2413
Epoch 705/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2417
Epoch 706/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2410
Epoch 707/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2414
Epoch 708/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2414
Epoch 709/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2413
Epoch 710/1000
123627/123627 [==============================] - 5s 41us/step - loss: 0.2411
Epoch 711/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2414
Epoch 712/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2414
Epoch 713/1000
123627/123627 [==============================] - 5s 40us/step - loss: 0.2416
Epo

In [7]:
def make_ergo_train_df(df):    
    # the ERGO II implementation expected the following columns to be preset in the dataframe
    # even if they are not used
    df['va'] = pd.NA
    df['vb'] = pd.NA
    df['ja'] = pd.NA
    df['jb'] = pd.NA
    df['t_cell_type'] = pd.NA
    df['protein'] = pd.NA
    df['mhc'] = pd.NA

    # using "UNK" for identifier of missing CDR3α
    df['tcra'] = "UNK"
    
    df['tcrb'] = df['tcrb'].str.replace('O','X')
    df['peptide'] = df['peptide'].str.replace('O','X')

    return df

def make_ergo_test_df(df):    
    map_keys = {
    'tcrb': 'TRB',
    'peptide': 'Peptide',
    'sign': 'sign'
    }
    df = df.rename(columns={c: map_keys[c] for c in df.columns})

    # the ERGO II implementation expected the following columns to be preset in the dataframe
    # even if they are not used
    df['TRAV'] = pd.NA
    df['TRBV'] = pd.NA
    df['TRAJ'] = pd.NA
    df['TRBJ'] = pd.NA
    df['T-Cell-Type'] = pd.NA
    df['Protein'] = pd.NA
    df['MHC'] = pd.NA

    # using "UNK" for identifier of missing CDR3α
    df['TRA'] = "UNK"
    df['TRB'] = df['TRB'].str.replace('O','X')
    df['Peptide'] = df['Peptide'].str.replace('O','X')

    return df

In [8]:
results_ergo2 = []

df_train = make_ergo_train_df(vdjdb.copy())
df_test = make_ergo_test_df(mcpas.copy())

model = ERGO2(
    gpu=[0],
    use_alpha=False,
    random_seed=0,
    train_val_ratio=.2,
)
model.train(df_train, epochs=1000)
prediction_df = model.test(df_test)

scores_df = get_scores(
    y_true=prediction_df['sign'].to_numpy(), 
    y_prob=prediction_df['prediction'].to_numpy(),
    y_pred=prediction_df['prediction'].to_numpy().round(),
)
results_ergo2.append(scores_df)
df_test['prediction'] = prediction_df['prediction']
df_test.to_csv(RESULTS_BASE+f"ergo2.vdjdb2mcpas.csv", index=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/home/nle-fgrazioli/anaconda3/envs/tcrmodels/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name          | Type         | Params
-----------------------------------------------
0 | tcrb_encoder  | LSTM_Encoder | 3 M   
1 | pep_encoder   | LSTM_Encoder | 3 M   
2 | hidden_layer1 | Linear       | 31 K  
3 | relu          | LeakyReLU    | 0     
4 | o

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]